In [4]:
import sys

sys.path.append(
    "C:\\Users\\Wyss User\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages"
)

import biolib
import gzip
import math


import matplotlib.pyplot as plt

import numpy as np
import os
import pandas as pd
import requests


import xml.etree.ElementTree as ET

from io import StringIO
from IPython.display import Image
from matplotlib_venn import venn2
from scipy import stats
from scipy.stats import f_oneway

In [9]:
ALL_FASTA_OUTPUT_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\all_fasta.fasta"
ASSAY_LIST_PATH = (
    "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\assay-list-olink-explore-3072.xlsx"
)
BRAIN_RNA_SEQ_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\brain_rna_seq.csv"
HGNC_IDS = (
    "https://ftp.ebi.ac.uk/pub/databases/genenames/hgnc/tsv/hgnc_complete_set.txt"
)
ALL_FASTA_OUTPUT_PATH = (
    "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\all_fasta.fasta"
)
MANIFEST_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\Shah (BWH)_OLINK_SK-5T4V_2023-06-27_MN UPDATES 82823.xlsx"
PLATE_LAYOUT_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\Plate Layout.xlsx"


RAW_DATA_FILE = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\LCSET_28343_10-4-2023_EXTENDED_NPX_2023-10-20.csv"
UNIPROT_TO_FASTA_PATH = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\uniprotkb_proteome_UP000005640_2023_11_20.fasta.gz"


PANELS = [
    "Cardiometabolic",
    "Cardiometabolic_II",
    "Inflammation",
    "Inflammation_II",
    "Neurology",
    "Neurology_II",
    "Oncology",
    "Oncology_II",
]

UNIMPORTANT_COLUMNS = [
    "Index",
    "OlinkID",
    "Panel_Lot_Nr",
    "PlateID",
    "WellID",
    "Normalization",
    "Processing_StartDate",
    "Processing_EndDate",
    "AnalyzerID",
    "ExploreVersion",
]


X_AXIS_ORDER = [
    "Fract 6",
    "Fract 7",
    "Fract 8",
    "Fract 9",
    "Fract 10",
    "Fract 11",
    "Fract 12",
    "Fract 13",
    "Fract 14",
    "Fract 15",
    "CSF A Internal EV",
    "CSF B Internal EV",
    "CSF C Internal EV",
    "CSF D Internal EV",
    "Plas A Internal EV",
    "Plas B Internal EV",
    "Plas C Internal EV",
    "Plasma D Internal EV",
]

In [10]:
# import the assay list file and map the uniprot id's to fasta

assay_list = pd.read_excel(ASSAY_LIST_PATH)

In [13]:
def parse_gz_file(file_path):
    protein_dict = {}
    current_uniprot_id = None
    current_sequence = ""

    with gzip.open(file_path, "rt") as f:
        for line in f:
            line = line.strip()

            if line.startswith(">"):
                if current_uniprot_id is not None:
                    protein_dict[current_uniprot_id] = current_sequence
                    current_sequence = ""
                if "|" in line:
                    current_uniprot_id = line.split("|")[1].strip()
                else:
                    print(f"Skipping line without expected format: {line}")
                    current_uniprot_id = None
            else:
                current_sequence += line
        if current_uniprot_id is not None:
            protein_dict[current_uniprot_id] = current_sequence

    return protein_dict


protein_data = parse_gz_file(UNIPROT_TO_FASTA_PATH)

assay_list["Sequence"] = assay_list["UniProt ID"].map(
    lambda x: protein_data.get(x, "N/A")
)

# export the list of proteins in fasta format to be input into the online program
# link to program: https://dtu.biolib.com/DeepTMHMM
with open(ALL_FASTA_OUTPUT_PATH, "w") as output_file:
    for index, row in assay_list.iterrows():
        output_file.write(f'>{row["UniProt ID"]}\n')
        output_file.write(f'{row["Sequence"]}\n')

In [15]:
output_directory = "C:\\Users\\Wyss User\\Documents\\EVs\\OLINK\\tmhmm_all"

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

os.chdir(output_directory)

deeptmhmm = biolib.load("DTU/DeepTMHMM")

for (
    _,
    row,
) in (
    assay_list.iterrows()
):  # there has got to be a way to make this faster this is insanely slow
    sequence = row["Sequence"]
    uniprot_id = row["UniProt ID"]
    # !echo -e f">{uniprot_id}\n{sequence}\n" > query.fasta
    fasta_line = f">{uniprot_id}\n{sequence}\n"
    with open("query.fasta", "w") as fasta_file:
        fasta_file.write(fasta_line)
    deeptmhmm_job = deeptmhmm.cli(args="--fasta query.fasta")
    print(deeptmhmm_job)
    deeptmhmm_job.save_files(output_directory)

2023-11-29 18:49:25,060 | INFO : Loaded project DTU/DeepTMHMM:1.0.24
2023-11-29 18:49:28,138 | INFO : Cloud: Initializing
2023-11-29 18:49:30,893 | INFO : Cloud: Pulling images...
2023-11-29 18:49:30,894 | INFO : Cloud: Computing...
Running DeepTMHMM on 1 sequence...
Step 1/4 | Loading transformer model...

Step 2/4 | Generating embeddings for sequences...
Generating embeddings: 100% 1/1 [00:00<00:00, 13.49seq/s]

Step 3/4 | Predicting topologies for sequences in batches of 1...
Topology prediction: 100% 1/1 [00:00<00:00,  3.64seq/s]

Step 4/4 | Generating output...
2023-11-29 18:49:51,517 | INFO : Cloud: Computation finished
2023-11-29 18:49:51,519 | INFO : Cloud: Result Ready
2023-11-29 18:49:51,520 | INFO : Waiting for job 951c63ee-4415-494a-ace5-06580c6d1f4c to finish...
2023-11-29 18:49:52,654 | INFO : Job 951c63ee-4415-494a-ace5-06580c6d1f4c has finished.
Job for DTU/DeepTMHMM:1.0.24 created at 2023-11-29T23:49:25.483170Z (951c63ee-4415-494a-ace5-06580c6d1f4c)
2023-11-29 18:49:55